In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import os

## Read data 

### Load XSUM data

In [3]:
xsum_dirpath = "data/xsum"
if os.path.exists(xsum_dirpath):
    print("XSUM Dataset found! Loading from path...")
    dataset = load_from_disk(xsum_dirpath)
else:
    print(f"XSUM Dataset not found! Downloading and saving to {xsum_dirpath}")
    dataset = load_dataset("EdinburghNLP/xsum")
    dataset.save_to_disk(xsum_dirpath)

XSUM Dataset found! Loading from path...


In [4]:
# Shuffle data
seed = 42
dataset = dataset.shuffle(seed=seed)
len(dataset["train"])

In [139]:
# Select a sample of 5000 articles for now
n_samples = 5000
samples = dataset["train"].select(range(n_samples))

### Load hewlett data (In progress)

In [6]:
# filepath = "/home/dafirebanks/projects/CS567_ML_Project/data/hewlett-foundation-data/training_set_rel3.xlsx"
# df = pd.read_excel(filepath)
# df.head()

KeyboardInterrupt: 

In [ ]:
# # Map essay sets to the actual prompts:
# set2prompt = {
#     1: "More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.",
#     2: """Censorship in the Libraries
# "All of us can think of a book that we hope none of our children or any other children have taken off the shelf. But if I have the right to remove that book from the shelf -- that work I abhor -- then you also have exactly the same right and so does everyone else. And then we have no books left on the shelf for any of us." --Katherine Paterson, Author
# Write a persuasive essay to a newspaper reflecting your vies on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.""",
#     3: """Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.
# Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.""",
#     4: """We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part.""",
# }

In [ ]:
# df.columns

In [ ]:
# # Selecting only the columns that are needed: essay_set, essay_id and essay
# df = df[["essay_set", "essay_id", "essay"]]
# df.head()

,essay_set,essay_id,essay
0,1,1,"Dear local newspaper, I think effects computer..."
1,1,2,"Dear @CAPS1 @CAPS2, I believe that using compu..."
2,1,3,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl..."
3,1,4,"Dear Local Newspaper, @CAPS1 I have found that..."
4,1,5,"Dear @LOCATION1, I know having computers has a..."


### Filter out essays with illegible words


## Setup prompts

In [11]:
xsum_prompts = [sample["document"] for sample in samples]
xsum_prompts[0]

"In Wales, councils are responsible for funding and overseeing schools. But in England, Mr Osborne's plan will mean local authorities will cease to have a role in providing education. Academies are directly funded by central government and head teachers have more freedom over admissions and to change the way the"

In [140]:
xsum_prompts[0], len(xsum_prompts)

('In Wales, councils are responsible for funding and overseeing schools.\nBut in England, Mr Osborne\'s plan will mean local authorities will cease to have a role in providing education.\nAcademies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.\nIt is a significant development in the continued divergence of schools systems on either side of Offa\'s Dyke.\nAnd although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.\nMinisters have no plans to follow suit.\nAt the moment, governing bodies are responsible for setting school hours and they need ministerial permission to make significant changes.\nThere are already more than 2,000 secondary academies in England and its extension to all state schools is unlikely to shake the Welsh Government\'s attachment to what they call a "community, comprehensive mod

## Setup model

In [145]:
model_name = "mistralai/Mistral-7B-v0.1"  # "gpt2-medium"
download_dir = "/home/dafirebanks/projects/dont-stop-prompting/models"
batch_size = 50
DEVICE = "cuda"

In [87]:
# del model
# del tokenizer

In [88]:
set_seed(42)
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=download_dir).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=download_dir)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Switch the padding side to the right - doesn't really matter because all texts should have at least 30 tokens, but this is to be consistent with the original code
if tokenizer.padding_side == "left":
    tokenizer.padding_side = "right"
tokenizer.padding_side

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

'left'

In [117]:
# NOTE: Taken from https://github.com/eric-mitchell/detect-gpt/blob/447d2ce8177004203f42d1da87d7f93a2e31ad52/run.py#L208 to ensure consistency


def encode_and_truncate(texts, n_prompt_tokens):
    """Encode each text as a list of token ids, and truncate to the first n_prompt_tokens tokens.
    Args:
        texts: list[str]
            List of texts to encode.
        n_prompt_tokens: int
            Number of tokens to keep for each text. If the text has fewer than this many tokens, it will be padded.
    Returns:
        dict[str, torch.Tensor]
            Dictionary of tensors, with keys "input_ids", "attention_mask", "token_type_ids".
    """
    all_encoded = tokenizer(texts, return_tensors="pt",
                            padding=True).to(DEVICE)
    return {key: value[:, :n_prompt_tokens] for key, value in all_encoded.items()}


def sample_from_model(texts, min_words=55, n_prompt_tokens=30):
    """Sample from the model using the provided texts as prompts, until each sample has at least min_words words.
    Args:
        texts: list[str]
            List of texts to use as prompts.
        min_words: int
            Minimum number of words for each sample.
        n_prompt_tokens: int
            Number of tokens to use for each prompt.
    Returns:
        list[str]
            List of samples generated from the model.
    """
    all_encoded = encode_and_truncate(texts, n_prompt_tokens)
    decoded = ["" for _ in range(len(texts))]
    sampling_kwargs = {
        "max_length": 200,
        "do_sample": True,
        "top_k": 40,
        "top_p": 0.96,
        "min_length": 150,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }

    # sample from the model until we get a sample with at least min_words words for each example
    # this is an inefficient way to do this (since we regenerate for all inputs if just one is too short), but it works
    tries = 0
    while (m := min(len(x.split()) for x in decoded)) < min_words:
        if tries != 0:
            print()
            print(
                f"min words: {m}, needed {min_words}, regenerating (try {tries})")

        outputs = model.generate(**all_encoded, **sampling_kwargs)
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        tries += 1

    return decoded


def trim_to_shorter_length(texta, textb):
    # truncate to shorter of o and s
    shorter_length = min(len(texta.split(" ")), len(textb.split(" ")))
    texta = " ".join(texta.split(" ")[:shorter_length])
    textb = " ".join(textb.split(" ")[:shorter_length])
    return texta, textb


def generate_samples(raw_data, batch_size):
    """
    NOTE: Samples are truncated to the length of the shorter text. This is to ensure that the samples are comparable.
    """
    data = {
        "original": [],
        "sampled": [],
    }

    for batch in range(len(raw_data) // batch_size):
        print("Generating samples for batch", batch,
              "of", len(raw_data) // batch_size)
        original_text = raw_data[batch * batch_size: (batch + 1) * batch_size]
        sampled_text = sample_from_model(original_text, min_words=55)

        for o, s in zip(original_text, sampled_text):
            o, s = trim_to_shorter_length(o, s)

            # add to the data
            data["original"].append(o)
            data["sampled"].append(s)

    return data

## Test generations

In [141]:
test = generate_samples(xsum_prompts, batch_size=batch_size)

Generating samples for batch 0 of 100


Generating samples for batch 1 of 100
Generating samples for batch 2 of 100
Generating samples for batch 3 of 100
Generating samples for batch 4 of 100
Generating samples for batch 5 of 100
Generating samples for batch 6 of 100
Generating samples for batch 7 of 100
Generating samples for batch 8 of 100
Generating samples for batch 9 of 100
Generating samples for batch 10 of 100
Generating samples for batch 11 of 100
Generating samples for batch 12 of 100
Generating samples for batch 13 of 100


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generating samples for batch 14 of 100
Generating samples for batch 15 of 100
Generating samples for batch 16 of 100
Generating samples for batch 17 of 100
Generating samples for batch 18 of 100
Generating samples for batch 19 of 100
Generating samples for batch 20 of 100
Generating samples for batch 21 of 100
Generating samples for batch 22 of 100
Generating samples for batch 23 of 100
Generating samples for batch 24 of 100
Generating samples for batch 25 of 100
Generating samples for batch 26 of 100
Generating samples for batch 27 of 100
Generating samples for batch 28 of 100
Generating samples for batch 29 of 100
Generating samples for batch 30 of 100
Generating samples for batch 31 of 100
Generating samples for batch 32 of 100
Generating samples for batch 33 of 100
Generating samples for batch 34 of 100
Generating samples for batch 35 of 100
Generating samples for batch 36 of 100


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generating samples for batch 37 of 100
Generating samples for batch 38 of 100
Generating samples for batch 39 of 100
Generating samples for batch 40 of 100
Generating samples for batch 41 of 100
Generating samples for batch 42 of 100
Generating samples for batch 43 of 100
Generating samples for batch 44 of 100
Generating samples for batch 45 of 100
Generating samples for batch 46 of 100
Generating samples for batch 47 of 100
Generating samples for batch 48 of 100
Generating samples for batch 49 of 100
Generating samples for batch 50 of 100
Generating samples for batch 51 of 100
Generating samples for batch 52 of 100
Generating samples for batch 53 of 100
Generating samples for batch 54 of 100
Generating samples for batch 55 of 100
Generating samples for batch 56 of 100
Generating samples for batch 57 of 100
Generating samples for batch 58 of 100
Generating samples for batch 59 of 100
Generating samples for batch 60 of 100
Generating samples for batch 61 of 100
Generating samples for ba

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generating samples for batch 74 of 100
Generating samples for batch 75 of 100
Generating samples for batch 76 of 100
Generating samples for batch 77 of 100
Generating samples for batch 78 of 100
Generating samples for batch 79 of 100
Generating samples for batch 80 of 100
Generating samples for batch 81 of 100
Generating samples for batch 82 of 100
Generating samples for batch 83 of 100
Generating samples for batch 84 of 100
Generating samples for batch 85 of 100
Generating samples for batch 86 of 100
Generating samples for batch 87 of 100
Generating samples for batch 88 of 100
Generating samples for batch 89 of 100
Generating samples for batch 90 of 100
Generating samples for batch 91 of 100
Generating samples for batch 92 of 100
Generating samples for batch 93 of 100
Generating samples for batch 94 of 100
Generating samples for batch 95 of 100
Generating samples for batch 96 of 100
Generating samples for batch 97 of 100
Generating samples for batch 98 of 100
Generating samples for ba

In [ ]:
# Batches to look into in case they may have weird padding: 13, 36, 73

In [144]:
len(test["sampled"])

5000

In [153]:
# Look at some generated samples
j = 20
i = 13 * batch_size + j
test["original"][i]

"Jeremy Corbyn unusually had the better of Theresa May in Prime Minister's Questions, brandishing leaked texts across the despatch box, claiming evidence that the Tories had given Surrey a special deal to avoid the chance of a damaging 15% council tax rise in a Conservative safe haven.\nThe council, and ministers, denied there had been any stitch-up.\nBut hours later, the government admitted they had agreed, in theory, that Surrey County Council could, like several others, try out keeping all of the business rates they raise from 2018, which could plug the gaps in funding in future.\nThat change is due to be in force across in England by 2020. Technically therefore, Surrey County Council has not been offered any additional funding. But the prospect of more flexibility over their own income in future could help fill the council's coffers, and seems to have eased some of their concerns.\nBut"

In [154]:
test["sampled"][i]

'Jeremy Corbyn unusually had the better of Theresa May in Prime Minister\'s Questions, brandishing leaked texts across the despatch box to taunt the Prime Minister about her own Brexit plan.\n\nThe PM was forced to defend plans to pay a £40 billion Brexit bill and was under pressure from Labour to reveal the Government\'s plan for future relations with the EU after Brexit.\n\nHowever, she was caught off guard by the release of documents that suggested that she believed a "soft" Brexit option was the only way to save the Union and "avert" a second Scottish independence referendum.\n\nThe documents were released by Labour as they questioned why the Government would choose an outcome which will "damage economic prospects, endanger our industrial base and cut living standards for the many to line the pockets of the few".\n\nOne document, a Downing Street briefing note for Mrs May, claimed a "soft"'

## Store it

In [155]:
# Turn into a jsonl format
all_data = []
for i in range(len(test["original"])):
    nongen = {"text": test["original"][i], "label": 0}
    gen = {"text": test["sampled"][i], "label": 1}

    all_data.append(nongen)
    all_data.append(gen)

len(all_data)

10000

In [161]:
# Visualize as a df
store_dirpath = "data"
df = pd.DataFrame.from_records(all_data)
df.head()

,text,label
0,"In Wales, councils are responsible for funding...",0
1,"In Wales, councils are responsible for funding...",1
2,"Up to 100,000 youngsters will be eligible for ...",0
3,"Up to 100,000 youngsters will be eligible for ...",1
4,Middlesbrough and Brighton face each other on ...,0


,text,label
0,"In Wales, councils are responsible for funding...",0
1,"In Wales, councils are responsible for funding...",1
2,"Up to 100,000 youngsters will be eligible for ...",0
3,"Up to 100,000 youngsters will be eligible for ...",1
4,Middlesbrough and Brighton face each other on ...,0


In [158]:
# Optionally split into train/test
train, test = train_test_split(df, test_size=0.2, random_state=42)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Store
train.to_json(f"{store_dirpath}/xsum-gen-train.jsonl", orient="records", lines=True)
test.to_json(f"{store_dirpath}/xsum-gen-test.jsonl", orient="records", lines=True)